In [31]:
import networkx as nx
import csv
import pandas as pd
import tqdm 

In [4]:
data_folder = "data/csv/"
file1 = "hashtag_usages_per_hour_9n_9ngranmarchaporlajusticia_weighted.txt"
file2 = "hashtag_usages_per_hour_noaltarifazo_ruidazonacional_weighted.txt"
graphs_folder = "graphs"

In [7]:
df = pd.read_csv(data_folder + file1, sep=' ')
df

,user,hashtag,hour,weight
0,5479912,1995,437040,1
1,7517622,2009,437036,1
2,7517622,12517,437002,1
3,8133272,387,436994,1
4,8133272,387,437006,1
...,...,...,...,...
219483,1193504467095822337,13027,437063,1
219484,1193504467095822337,13990,437064,1
219485,1193504467095822337,14517,437076,2
219486,1193504467095822337,16513,437063,1


In [36]:
# Creo red de usuarios que comparten hastags
G = nx.Graph()
df_users = df["user"].unique()
print("num users: ", df_users.shape[0])
G.add_nodes_from(df_users)
for user in tqdm.tqdm(df_users):
    df_user_hastag = df.loc[df["user"] == user]
    df_user_hastag = df_user_hastag["hashtag"]
    #df_user_hastag.columns[1] = "hashtag"
    #print(df_user_hastag)
    #print("Encontrados ", df_user_hastag.shape[0], "hashtags del usuario ", user)
    for hashtag in df_user_hastag:
        df_hashtag = df.loc[df["hashtag"] == hashtag]
        df_hashtag = df_hashtag["user"]
        #print(df_hashtag.shape[0] , " usuarios comentaron sobre el hastag ", hashtag)
        for us in df_hashtag:
            # TODO Si buscamos eficiencia se puede quitar y añadir todas las conexiones y luego simplemente quitar la propia
            if us != user:
                # TODO Es no dirigido asi que el orden da igual(?)
                if G.has_edge(user, us):
                    #print("A")
                    G[user][us]["weight"] += 1
                elif G.has_edge(us, user):
                    print("B")                    
                    G[us][user]["weight"] += 1
                else:
                    G.add_edge(user, us, weight = 1)
nx.write_gexf(G, graphs_folder + "users_sharing_hashtags.gexf")

num users:  8022


  0%|          | 0/8022 [00:00<?, ?it/s]

 45%|████▌     | 3639/8022 [14:25<18:18,  3.99it/s]   

In [ ]:
# Creo red de hastags que comparten usuarios
# Creo red de usuarios que comparten hastags
G = nx.Graph()
df_hashtags = df["hashtag"].unique()
print("num hashtags: ", df_hashtags.shape[0])
G.add_nodes_from(df_hashtags)
for hashtag in tqdm.tqdm(df_hashtags):
    df_hashtag_user = df.loc[df["hashtag"] == df_hashtags]
    df_hashtag_user = df_hashtag_user["user"]
    #df_user_hastag.columns[1] = "hashtag"
    #print(df_user_hastag)
    #print("Encontrados ", df_user_hastag.shape[0], "hashtags del usuario ", user)
    for user in df_hashtag_user:
        df_user = df.loc[df["user"] == user]
        df_user = df_hashtag["hashtag"]
        #print(df_hashtag.shape[0] , " usuarios comentaron sobre el hastag ", hashtag)
        for hash in df_user:
            # TODO Si buscamos eficiencia se puede quitar y añadir todas las conexiones y luego simplemente quitar la propia
            if hash != hashtag:
                # TODO Es no dirigido asi que el orden da igual(?)
                if G.has_edge(hashtag, hash):
                    #print("A")
                    G[hashtag][hash]["weight"] += 1
                elif G.has_edge(hash, hashtag):
                    print("B")                    
                    G[hash][hashtag]["weight"] += 1
                else:
                    G.add_edge(hashtag, hash, weight = 1)
nx.write_gexf(G, graphs_folder + "hashtags_sharing_users.gexf")